In [2]:
%matplotlib notebook

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os 
from tqdm import tqdm

import multiprocessing as mp

import time 
from natsort import natsorted
import moviepy.video.io.ImageSequenceClip

import copy 

import pickle as pkl

In [4]:
sys.setrecursionlimit(10000) 

In [5]:
from simulation import *
from visualizer import renderParticles, drawTree, plotter

In [6]:
plt.ioff()

In [32]:
def f_multipole(p, node, L, theta=0.5, epsilon=1e-3, m_scale=1e-3):  
    # check to see if particle is looking at itself,return 0 
    if node.mass is not None and sum((node.com - p)**2)**0.5 < 1e-10:
        return np.zeros(2) 
    
    # if a leaf node and has a particle, directly solve for gravity 
    if isinstance(node, leaf) and node.mass is not None:
        plotter(ax, node, 'lime', alpha=1)
        return np.zeros(2)
    # if a leaf node and doesn't have a particle, return 0
    elif isinstance(node, leaf) and node.mass is None:
        return np.zeros(2)
    
    # if a quadtree that satisfies multipole condition, solve for gravity
    if isinstance(node, QuadTree) and node.length[0] / sum((node.com - p)**2)**0.5 <= theta: 
        plotter(ax, node, 'lime', alpha=1)
        return np.zeros(2)

        # return solve_acc(p, node, L, epsilon=epsilon, m_scale=m_scale)
    # else if quadtree and hasn't satisfied, recursively search the tree's children 
    elif isinstance(node, QuadTree) and node.length[0] / sum((node.com - p)**2)**0.5 > theta: 
        return sum(np.array([f_multipole(p, ch, L, theta=theta, epsilon=epsilon, m_scale=m_scale) for ch in [node.ll, node.ur, node.lr, node.ul]]))

In [88]:
L = 1
n = 1000
# particles = np.random.random((n, 4)) * L - L/2
particles[0, :2] = 0.0
# particles1 = np.random.normal(0.95, 0.25, (int(n / 2), 4))
# particles2 = np.random.normal(-0.75, 0.25, (int(n / 2), 4))
# particles2[:, 0] += 0.1
# particles = np.vstack((particles1, particles2))

# particles = np.random.random((100, 4)) * 2 - 1

particles[:, 2:] = 0

# update acceleration block. construct the quadtree for force modeling 
root = leaf(bbox=np.array([[-L/2, -L/2], [L/2, L/2]]))
# which we then can immediately turn into a tree 
for p in particles[:, 0:2]: 
    root = assignParticle(p, root)


    
for t, theta in zip(range(len(thetas)), thetas):
    fig, ax =plt.subplots(1, 1, figsize=(6, 6))
    ax.set_facecolor("black")

    # plt.scatter(particles[:, 0], particles[:, 1], s= 0.1)
    #plt.scatter(particles_list[-1][:, 0], particles_list[-1][:, 1], s=0.1)
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.6, 0.6)

    drawTree(ax, root)

    f_multipole(particles[0, 0:2], root, root.length[0], theta=theta) 
    plt.plot(particles[0, 0], particles[0, 1], marker='*', c='white')
    ax.annotate(r'$\theta=${:.2f}'.format(theta), xy=(-0.59, -0.59), c='white')
    plt.savefig('./thetas/frame_{}.png'.format( t), bbox_inches='tight', pad_inches = 0, dpi=300)
    plt.close()

# tpoints, particles_list, trees = leapfrog(particles.copy(), t_start=0, t_end=10, N=1, L=1, theta=0.5,
#                                          epsilon=1e-3, m_scale=1e3, multi=False)


In [82]:
thetas = np.linspace(0, 2, 41)
thetas = np.hstack((thetas[::-1], thetas))

In [81]:
thetas

82

In [89]:

tstart = time.time() 

# print('[{:.2f}] Creating plot directory ...'.format(time.time() - tstart))
# plot_dir = '{}/{}'.format(os.path.abspath(os.getcwd()), 'L{}n{}'.format(trees[0].length[0], n))
# if not os.path.isdir(plot_dir):
#     os.mkdir(os.path.join(plot_dir))

# print('[{:.2f}] Initializing multiprocessing ...'.format(time.time() - tstart))
# n_cpu = mp.cpu_count()
# pool = mp.Pool(processes=n_cpu)

# mp_trees = np.array_split(trees, n_cpu)
# mp_particles = np.array_split(particles_list, n_cpu)
# mp_tpoints = np.array_split(np.arange(len(tpoints)), n_cpu)
    
# print('[{:.2f}] Creating plots ...'.format(time.time() - tstart))
# for i in range(n_cpu):
#     pool.apply_async(renderParticles, args=(mp_particles[i], mp_trees[i], mp_tpoints[i], plot_dir, 300))

# pool.close()
# pool.join()
print('[{:.2f}] Multiprocess concluded ...'.format(time.time() - tstart))


print('[{:.2f}] Creating video ...'.format(time.time() - tstart))

plot_dir = './thetas/'
fps=2 #number of frames per second
image_files = natsorted([os.path.join(plot_dir,img) for img in os.listdir(plot_dir) if img.endswith(".png")], reverse=False)
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('test.mp4')

# print('[{:.2f}] Video created ...'.format(time.time() - tstart))

# _ = [os.remove(image_file) for image_file in image_files]
# os.rmdir(plot_dir)
# print('[{:.2f}] Directory cleaned ...'.format(time.time() - tstart))


[0.00] Multiprocess concluded ...
[0.00] Creating video ...
Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


In [ ]:
with open('./data/sim.pkl', 'wb') as f:  # open a text file
    pkl.dump({'tpoints': tpoints, 'particles_list': particles_list, 'trees': trees}, f) # serialize the list
f.close()

In [ ]:
with open('./data/sim.pkl', 'rb') as f:
    sim_dict = pkl.load(f)
    
tpoints = sim_dict['tpoints']
particles_list = sim_dict['particles_list']
trees = sim_dict['trees']

n = particles_list[0].shape[0]